In [1]:
# Installing libraries
# !pip install datasets evaluate transformers[sentencepiece]
# !pip install faiss-cpu
# !pip install datasets
# !pip install sentence-transformers
##!nvidia-smi 

In [2]:
import time

start = time.time()

In [3]:
# Importing library
import pandas as pd
import re
import time
from tqdm import tqdm
import seaborn as sns
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import nltk
from sqlalchemy import create_engine
from sqlalchemy import types
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize
import os
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

[nltk_data] Downloading package punkt to C:\Users\Dishita
[nltk_data]     Neve\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dishita
[nltk_data]     Neve\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Dishita
[nltk_data]     Neve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# # Reading April-June csv
# a = pd.read_csv(r'/content/gdrive/MyDrive/iBas-Data/iBAS-Apr-June-2022-Ticket Search Results.csv')
# print(a.shape)
# # Reading Jan-March csv
# b = pd.read_csv(r'/content/gdrive/MyDrive/iBas-Data/iBAS-Jan-Mar-2022-Ticket Search Results.csv')
# print(b.shape)
# # Reading July-August csv
# c = pd.read_csv(r'/content/gdrive/MyDrive/iBas-Data/iBAS-July-Aug-2022-Ticket Search Results.csv')
# print(c.shape)

from dotenv import load_dotenv
## load the credentials here
def load_envFile():
    try:
        ## load the dotenv
        load_dotenv()

        ## fetch the credentials
        host = os.environ["host"] 
        user = os.environ["user"]
        passwd = os.environ["passwd"]
        db_name = os.environ["db_name"]
        port = os.environ["port"]

    except Exception as e:
        print(f'{e}')

    else:
        return (host, user, passwd, db_name, port)

In [5]:
def pullDataFromDb(tbName, host, user, passwd, db_name, port):
    
    try:
        ## create the connection
        host = host
        user = user
        passwd = passwd
        db_name = db_name
        port = port

        engine = create_engine('postgresql://'+user+':'+passwd+'@'+host+':'+str(port)+'/'+db_name, echo=False)

        res = engine.execute(f'SELECT * FROM public."{tbName}"')

        result = res.fetchall()

        data = pd.DataFrame(result)
        data.columns = result[0].keys()

    except Exception as e:
        logging.exception("Exception occurred")
    
    else:
        return data


In [6]:
## get db creds
host, user, passwd, db_name, port = load_envFile()
tbName1 = 'AutotaskTicketData'
df = pullDataFromDb(tbName1, host, user, passwd, db_name, port)
#df = df.drop(['Id'], axis=1)

In [7]:
df

,ID,Classification,Ticket_Number,Ticket_Title,Description,Account,Queue,Resources,Status,Priority,Created,Due,Impact,Issue_Type,Sub_Issue_Type,Resolution
0,0,CDS-Northwest,T20220331.1024,Office 365 Licensing Change 3-28-2022 CHCS Ser...,Which Licenses need to be changed? (repeat che...,"iBAS, Inc (CHCS)",Billing,"ZBOROWSKI, STACEY (primary)",Complete,P5 - Service Request,2022-04-01,2022-04-08,None,Collaboration,MACD O365 Licensing Change,confirmed CSP BOSS contract updated appropriately
1,1,CDS-Northwest,T20220331.1078,fax xmedius fax,FROM: carol.morris@chcs-services.com\nFROM NAM...,"iBAS, Inc (CHCS)",CDS-Northwest,"BAGRI, PRABHJOT KAUR (primary)",Complete,P5 - Service Request,2022-04-01,2022-04-12,Single User,User Administration,Account Administration,None
2,2,CDS-Northwest,T20220331.1112,RE: To avoid business impact: need connectivit...,FROM: ken.cook@chcs-services.com\nFROM NAME: K...,"iBAS, Inc (CHCS)",Merged Tickets,None,Complete,Scheduled,2022-04-01,2022-04-01,None,None,None,None
3,3,CDS-Northwest,T20220331.1202,Password Reset Request,Password Reset Request\nCb-8502214024,"iBAS, Inc (CHCS)",Global Shared Service Desk,"PATODI, ANKIT (primary)",Complete,P3 - Moderate,2022-04-01,2022-04-02,Single User,User Administration,Password reset,None
4,4,CDS-Northwest,T20220331.1319,RFQ: Asset Tags for iBAS (CHCS) Hardware,"From: Tim Becker\nSent: Monday, March 28, 2022...","iBAS, Inc (CHCS)",Business Development,"LAWLER, COREY (primary)",Complete,Scheduled,2022-04-01,2022-05-01,Single User,:Service - Procurement,Procurement - New,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12550,3141,CDS-Northwest,T20220831.0836,LogicMonitor (error) Alert - CHUSE1DEVNPS01 - CPU,"Group: Devices by Type/Windows Servers, Client...","iBAS, Inc (CHCS)",EOC,"RICKARD, MATTHEW (primary)",Complete,P3 - Moderate,2022-08-31,2022-09-01,Multiple Users,Server Management,Storage Utilization,"Alert cleared, utilization was driven by (TiWo..."
12551,3142,CDS-Northwest,T20220831.0899,Office 365 Licensing Change - 08-31-2022 CHCS...,Which Licenses need to be changed? (repeat che...,"iBAS, Inc (CHCS)",Billing,"ZBOROWSKI, STACEY (primary)",MAC:RTB,P5 - Service Request,2022-08-31,2022-09-08,None,Collaboration,MACD O365 Licensing Change,None
12552,3143,CDS-Northwest,T20220831.0972,Laptop frozen,CB: 251-209-0563\n\nUser informed that the lap...,"iBAS, Inc (CHCS)",Global Shared Service Desk,"EACHAMBADI, NAUNIT (primary)",Complete,P3 - Moderate,2022-08-31,2022-09-01,Single User,Endpoint Management,Operating System - Issue,The laptop was plugged into the docking statio...
12553,3144,CDS-Northwest,T20220831.0975,FW: Email Account Will Be Closed,FROM: miriam.charles@chcs-services.com\nFROM N...,"iBAS, Inc (CHCS)",Global Shared Service Desk,"SONI, JITENDRA (primary)",Complete,P3 - Moderate,2022-08-31,2022-09-01,Single User,Software,:Other,None


In [8]:
# Function to identify REPLY emails
def replacing1(text) :
    count = 0
    removed = []
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] == 'RE' or tokens[i] == 're' or tokens[i] == 'Re':
            #print('ok')
            removed.append('1')
        else :
            continue
            return " ".join(removed) 

df['Ticket_Title_1'] = df['Ticket_Title'].apply(lambda x: replacing1(x))  

In [9]:
#df = df.head(2)

In [10]:
names = []
# Extracting name from email id
def extract_name_email(text) :
    e = []
    text = str(text)
    tokens = word_tokenize(text)
    lst = re.findall('\S+@\S+', text)   
    regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    for i in lst :
        if(re.fullmatch(regex, i)):
            x = i.split("@")
            names.extend(x)
            #print('x',x)
            y = i.split(".")
            names.extend(y)
            #print('y',y)
            if len(y) != 0 :
                for l in y :
                    z = l.split("@")
                    names.extend(z)
                    #print('z',z)

    n = list(set(names))
    #print('n',n)
    for i in range(len(tokens)) :
        if tokens[i] in n :
            continue
    else :
        e.append(tokens[i])
  #print(e)
    return " ".join(e)

In [11]:
# Customize email -- removing subject, regards
def tok(text):  
    text = str(text)
    tokens = word_tokenize(text)
    #print(tokens)
    c = []
    removed = []
    for i in range(len(tokens)):
        #print(tokens[i])
        if (i+2 < len(tokens)) or (i < len(tokens)) or (i+1 < len(tokens)) :
          #print("(i+2 < len(tokens)) or (i < len(tokens)) or (i+1 < len(tokens))",i+2,i,i+1,tokens[i])
          try :
            if (tokens[i] == 'from' and tokens[i+1] == ':') or (tokens[i]==':' and tokens[i-1] == 'from') or (tokens[i] == 'from' and tokens[i+1] == 'name') :
              continue
            elif (tokens[i] == 'email' and tokens[i+1] == 'body') or (tokens[i-1] == 'email' and tokens[i] == 'body'):
              continue
            elif ((tokens[i] == 'hi' or tokens[i] == 'hello') and (tokens[i+1] == 'team')) or (tokens[i] == 'ty'):
              continue
            elif (tokens[i] == 'good'  and tokens[i+1] == 'morning') or (tokens[i-1] == 'good'  and tokens[i] == 'morning') :
              continue
            elif (tokens[i] == 'hello'  and tokens[i+1] == ',') or (tokens[i] == 'hey') or (tokens[i] == 'name'  and tokens[i+1] == ':') :
              continue
            elif ((tokens[i-1] == 'hi' or tokens[i-1] == 'hello') and (tokens[i] == 'team')) :
              continue
            elif (tokens[i-1] == 'get' and tokens[i] == 'outlook' and tokens[i+1] == 'for' ) :
              continue
            elif (tokens[i] == 'to' and tokens[i+1] == ':') or (tokens[i] == 'cc'):
              continue
            elif (tokens[i] =='cid') :
              break
            elif (tokens[i] == 'regards' or tokens[i] == 'regard' or tokens[i] == 'thanks' or tokens[i] == 'thank') and (tokens[i+1] != 'for') or (tokens[i] == 'get' and tokens[i+1] == 'outlook' and tokens[i+2] == 'for') or (tokens[i] == 'regards' and tokens[i+1] == ','):
              break
            #elif (tokens[i] == 'get' and tokens[i+1] == 'outlook' and tokens[i+2] == 'for') or (tokens[i] == 'regards' and tokens[i+1] == ','):
             # break
            else :
              c.append(tokens[i])
          except :
            continue
        elif (i+2 > len(tokens)) or (i == len(tokens)) or (i+1 > len(tokens)) :
          c.append(tokens[i])
    
    if ('<' in c) and ('>' in c) :
      l = c.index('<')
      j = c.index('>')
      t = c[:l]
      u = c[j+1:]
      c = t + u
      ##print('if')
        
    return " ".join(c)


# Merging code of Software names
def remove_numbers(text):
    text = str(text)
    number_pattern = r'[~^0-9]'
    tokens = word_tokenize(text)
    #print(tokens)
    #print("len(tokens)",len(tokens))
    t = []
    for i in range(len(tokens)):
        try :
            if (i == len(tokens)-1) :
              #print("(i == len(tokens)-1)",i)
              t.append(tokens[i])
            elif (tokens[i] == 'office' or  tokens[i] =='office365') and (tokens[i+1] == '365') :
                #print('office365')
                t.append('office365')
            elif (tokens[i] == 'bau') and (tokens[i+1] =='360') :
                t.append('bau360')
            elif (tokens[i] == 'forti' and tokens[i+1] == 'client') :
                #print('forticlient')
                t.append('forticlient')
            elif (tokens[i]=='non') and (tokens[i+1] == 'prod') :
                t.append('nonprod')
            elif (tokens[i] == 'as' and tokens[i+1] == '400') or (tokens[i] == 'as400') :
                #print('as400')
                t.append('as400')    
            elif (tokens[i]== 'e3') or (tokens[i]== 'e3') or (tokens[i]== 'ec2') or (tokens[i]== 'develop03') or (tokens[i]== 'ie5') or (tokens[i]== 'ie10') or (tokens[i]== 'as400'):
                t.append(tokens[i])
            elif (tokens[i]== 'case') or (tokens[i+1]== '360') or (tokens[i]== 'case360') :
                #print('case360')
                t.append('case360')
            elif (tokens[i]== 'db' or tokens[i]== 'data') and (tokens[i+1]== 'base'):
                t.append('database')
            elif (tokens[i]=='non') and (tokens[i]=='prod') :
                t.append('nonprod')
            elif (tokens[i]== 'ms') :
                t.append('microsoft')
            elif (tokens[i]== 'screen') or (tokens[i]== 'shot') :
                t.append('screenshot')
            elif (tokens[i] == 'do') and (tokens[i] == 'nt') :
                t.append('do')
                t.apppend('not')
            elif (tokens[i] == 'pfa') :
                #print('pfa')
                t.append('please')
                t.append('find')
                t.append('attachment')
            elif (tokens[i] == 'eod') :
                #print('eod')
                t.append('end')
                t.append('of')
                t.append('day')
            elif (tokens[i] == 'pw') :
                #print('eod')
                t.append('password')
            elif (tokens[i]== 'five9') or (tokens[i]=='five') or (tokens[i+1]=='9') :
                t.append('five9')
            elif (tokens[i]=='vpn') :
                t.append('vpn')
            elif (tokens[i]== 'app') :
                t.append('application')
            elif (tokens[i]=='vm') :
                t.append('virtual')
                t.append('machine')
            elif (tokens[i]=='appo6') :
                t.append('appo6')
            elif (tokens[i]== 'PW' or tokens[i]=='pw' or tokens[i]=='password') :
                t.append('password')
            elif (tokens[i] == 'tciket') :
                t.append('ticket')
            elif (tokens[i] == 'ca') and (tokens[i] == 'nt') :
                t.append('cannot')
            ## else :
                #without_number = re.sub(pattern=number_pattern, repl="", string=tokens[i])
                #t.append(str(without_number))
            elif tokens[i].isnumeric() and (t[-1] == 'appo6' or t[-1] == 'office365' or t[-1] == 'five9' or t[-1] == 'case360' or t[-1] == 'as400' or t[-1] == 'ec2' or t[-1] == 'bau360' or t[-1] == 'e3' or t[-1] == 'ie5' or t[-1] == 'ie10') :
                continue
            elif (tokens[i] == 'from' and tokens[i+1] == 'name') or (tokens[i-1] == 'from' and tokens[i] == 'name'):
                continue
            else :
                t.append(tokens[i])
              #print("t.append(tokens[i])",tokens[i])
        except :
            continue
    #print(t)
    return " ".join(t)



In [12]:
# Removing function
import string
def remove_punctuation(text):
    # Make a translation table to remove all punctuation symbols
    translator = text.maketrans('', '', string.punctuation)
    # Use the translation table to remove all punctuation symbols from the text
    return text.translate(translator)

def remove_urls(text):
    # Use a regular expression to search for URLs
    url_pattern = r'https?://\S+'
    return re.sub(url_pattern, '', text)

def re_num_tt(text) :
    number_pattern = r'[~^0-9]'
    return re.sub(number_pattern, '', text)

# Converting text to lower case
def convert_to_lower(text):
    text = str(text)
    return text.lower()

def remove_voice_questions(text) :
    text = str(text)
    tokens = word_tokenize(text)
    t = []
    c = ['nt','know','me']
    for i in range(len(tokens)) :
        t.append(tokens[i])

    for j in range(len(tokens)) :
        if t[0] == 'your' :
            index = t.index('impacted')
            t = t[index+2:]
            break
#     else :
#         continue
    if t is None :
        t = []
        t.append('Empty')

In [13]:
# Replacing office 365 to office365
df['Description_lower'] = df['Description'].apply(lambda x: convert_to_lower(x))   
df['Description_email'] = df['Description_lower'].apply(lambda x: extract_name_email(x))
df['Description_url'] = df['Description_email'].apply(lambda x: remove_urls(x)) 
df['Description_tok'] = df['Description_url'].apply(lambda x: tok(x))
df['Description_punc'] = df['Description_tok'].apply(lambda x: remove_punctuation(x))
df['Description_digits'] = df['Description_punc'].apply(lambda x: remove_numbers(x)) 
##df['Description_1'] = df['Description_digits'].apply(lambda x: remove_voice_questions(x))   

In [14]:
df['Ticket_Title'] = df['Ticket_Title'].apply(lambda x: convert_to_lower(x))   
df['Ticket_Title'] = df['Ticket_Title'].apply(lambda x: tok(x))
df['Ticket_Title'] = df['Ticket_Title'].apply(lambda x: remove_numbers(x)) 
df['Ticket_Title'] = df['Ticket_Title'].apply(lambda x: re_num_tt(x)) 

In [15]:
df['Description_email'][1]

'.'

In [16]:
#df['Description_tok'][1]

In [17]:
#df['Description_digits'][1]

In [18]:
print(df.shape)
df.drop_duplicates(subset='Ticket_Number', keep="first")
print(df.shape)

(12555, 23)
(12555, 23)


In [19]:
print(df.shape)
df =df[df['Ticket_Title_1'] = '']
print(df.shape)

In [20]:
# Merging ticket title and description
df['Description_re_concate'] = df['Description_digits'] + ' ' + df['Ticket_Title']

In [21]:
df.to_csv('Text_Processing.csv',index= False)

In [22]:
# Pre trained Sentence Transformers
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [23]:
import torch

#device = torch.device("cuda")
device = torch.device("cpu")
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0): MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_features

In [24]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
    text_list, padding=True, truncation=True, return_tensors="pt")
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [25]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(df)
comments_dataset

Dataset({
    features: ['ID', 'Classification', 'Ticket_Number', 'Ticket_Title', 'Description', 'Account', 'Queue', 'Resources', 'Status', 'Priority', 'Created', 'Due', 'Impact', 'Issue_Type', 'Sub_Issue_Type', 'Resolution', 'Ticket_Title_1', 'Description_lower', 'Description_email', 'Description_url', 'Description_tok', 'Description_punc', 'Description_digits', 'Description_re_concate'],
    num_rows: 12555
})

In [26]:
embedding = get_embeddings(comments_dataset['Description_re_concate'][0])
embedding.shape

torch.Size([1, 768])

In [27]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["Description_re_concate"]).detach().cpu().numpy()[0]}
)

  0%|          | 0/12555 [00:00<?, ?ex/s]

In [28]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/13 [00:00<?, ?it/s]

Dataset({
    features: ['ID', 'Classification', 'Ticket_Number', 'Ticket_Title', 'Description', 'Account', 'Queue', 'Resources', 'Status', 'Priority', 'Created', 'Due', 'Impact', 'Issue_Type', 'Sub_Issue_Type', 'Resolution', 'Ticket_Title_1', 'Description_lower', 'Description_email', 'Description_url', 'Description_tok', 'Description_punc', 'Description_digits', 'Description_re_concate', 'embeddings'],
    num_rows: 12555
})

In [29]:
embeddings_dataset

Dataset({
    features: ['ID', 'Classification', 'Ticket_Number', 'Ticket_Title', 'Description', 'Account', 'Queue', 'Resources', 'Status', 'Priority', 'Created', 'Due', 'Impact', 'Issue_Type', 'Sub_Issue_Type', 'Resolution', 'Ticket_Title_1', 'Description_lower', 'Description_email', 'Description_url', 'Description_tok', 'Description_punc', 'Description_digits', 'Description_re_concate', 'embeddings'],
    num_rows: 12555
})

In [30]:
import pickle
# Save the text embeddings to a file using pickle
with open("text_embeddings.pkl", "wb") as f:
    pickle.dump(embeddings_dataset, f)

### Sample Questions for testing

In [31]:
question = "documents not opening in case360"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [32]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=20
)

In [33]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores

In [37]:
from numpy.linalg import norm
cs = []
for i in samples_df['embeddings'] :
    cosine = np.dot(question_embedding,i)/(norm(question_embedding)*norm(i))
    cs.append(cosine)

In [38]:
samples_df["Cosine_Score"] = cs
samples_df.sort_values("Cosine_Score", ascending=False, inplace=True)

samples_df5 = samples_df.head(5)
t = [1,2,3,4,5]
samples_df5['No'] = t

C:\Users\Dishita Neve\AppData\Local\Temp\ipykernel_23336\2936664715.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples_df5['No'] = t


In [41]:
# Printing top 5 results in the output
for _, row in samples_df5.iterrows():
    print(f"**Serial Number** {row.No}")
    print(f"**TICKET TITLE:** {row.Ticket_Number}")
    print(f"**TITLE:** {row.Ticket_Title}")
    print(f"**CONFIDENCE SCORE(%):** {round(row.Cosine_Score[0],2)}")
    print("=" * 50)
    print()

**Serial Number** 1
**TICKET TITLE:** T20220720.1136
**TITLE:** case documents not opening
**CONFIDENCE SCORE(%):** 0.84

**Serial Number** 2
**TICKET TITLE:** T20220415.0809
**TITLE:** unable to open documents in case 
**CONFIDENCE SCORE(%):** 0.82

**Serial Number** 3
**TICKET TITLE:** T20220413.0404
**TITLE:** unable to open documents in case 
**CONFIDENCE SCORE(%):** 0.82

**Serial Number** 4
**TICKET TITLE:** T20220310.0485
**TITLE:** case documents not open
**CONFIDENCE SCORE(%):** 0.81

**Serial Number** 5
**TICKET TITLE:** T20220614.0637
**TITLE:** not able to view documents in case application
**CONFIDENCE SCORE(%):** 0.8

